# Getting Started With TensorFlow 

Source: https://www.tensorflow.org/get_started/get_started

TensorFlow provides multiple APIs. The lowest level API – TensorFlow Core – provides you with complete programming control. We recommend TensorFlow Core for machine learning researchers and others who require fine levels of control over their models. The higher level APIs are built on top of TensorFlow Core. These higher level APIs are typically easier to learn and use than TensorFlow Core. In addition, the higher level APIs make repetitive tasks easier and more consistent between different users. A high-level API like ``tf.estimator`` helps you manage data sets, estimators, training and inference.

## Tensors

The central unit of data in TensorFlow is the **tensor**. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's **rank** is its number of dimensions. Here are some examples of tensors:

In [1]:
3 # a rank 0 tensor; this is a scalar with shape []
[1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

# TensorFlow Core

The canonical import statement for TensorFlow programs is as follows:m

In [2]:
import tensorflow as tf

This gives Python access to all of TensorFlow's classes, methods, and symbols. Most of the documentation assumes you have already done this.

## The Computational Graph

You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph.
2. Running the computational graph.

A **computational graph** is a series of TensorFlow operations arranged into a graph of nodes. To do efficient numerical computing in Python, we typically use libraries like NumPy that do expensive operations such as matrix multiplication outside Python, using highly efficient code implemented in another language. Unfortunately, there can still be a lot of overhead from switching back to Python every operation. This overhead is especially bad if you want to run computations on GPUs or in a distributed manner, where there can be a high cost to transferring data.

TensorFlow also does its heavy lifting outside Python, but it takes things a step further to avoid this overhead. Instead of running a single expensive operation independently from Python, TensorFlow lets us describe a graph of interacting operations that run entirely outside Python.

Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a ``tf.constant``. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors ``node1`` and ``node2`` as follows:

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


## Sessions

Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect. Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. To actually evaluate the nodes, we must run the computational graph within a **session**.

TensorFlow relies on a highly efficient C++ backend to do its computation. The connection to this backend is called a **session**. The common usage for TensorFlow programs is to first create a graph and then launch it in a session. A session encapsulates the control and state of the TensorFlow runtime.

The following code creates a ``Session`` object and then invokes its run method to run enough of the computational graph to evaluate ``node1`` and ``node2``. By running the computational graph in a session as follows:

In [4]:
sess = tf.Session()
sess.run([node1, node2])

[3.0, 4.0]

we see the expected values of 3.0 and 4.0.

## Operations

We can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes). For example, we can add our two constant nodes and produce a new graph as follows:

In [5]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


TensorFlow provides a utility called **TensorBoard** that can display a picture of the computational graph. Here is a screenshot showing how TensorBoard visualizes the graph:

![Graph](https://www.tensorflow.org/images/getting_started_add.png)

As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The preceding three lines are a bit like a function or a lambda in which we define two input parameters (``a`` and ``b``) and then an operation on them. We can evaluate this graph with multiple inputs by using the ``feed_dict`` argument to the ``run`` method to feed concrete values to the placeholders:

In [7]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In TensorBoard, the graph looks like this:

![Graph](https://www.tensorflow.org/images/getting_started_adder.png)

We can make the computational graph more complex by adding another operation. For example,

In [8]:
add_and_triple = adder_node * 3.
sess.run(add_and_triple, {a: 3, b: 4.5})

22.5

The preceding computational graph would look as follows in TensorBoard:

![Graph](https://www.tensorflow.org/images/getting_started_triple.png)

## Variables and Placeholders

In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. **Variables** allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when you call ``tf.constant``, and their value can never change. By contrast, variables are not initialized when you call ``tf.Variable``. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [10]:
init = tf.global_variables_initializer()
sess.run(init)

This step takes the initial values (in this case tensors with values of ``3`` and ``-3``) that have already been specified, and assigns them to each `tf.Variable`.
It is important to realize ``init`` is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call ``sess.run``, the variables are uninitialized.

Since ``x`` is a placeholder, we can evaluate ``linear_model`` for several values of ``x`` simultaneously as follows:

In [11]:
sess.run(linear_model, {x: [1, 2, 3, 4]})

array([ 0.        ,  0.30000001,  0.60000002,  0.90000004], dtype=float32)